# Portfolio Optimization

We are given the following Cost function to minimize
$$C(\{x_i\}) = \sum_i b_i x_i  + \sum_{ij} A_{ij} x_i x_j$$
where $x_i \in \{0, 1\}$.

To Solve on a QC, we change to $x_i=(1-z_i)/2$
$$
\begin{split}
C(\{z_i\}) &= \frac{1}{2} \sum_i b_i - \frac{1}{2} \sum_i b_i z_i + \frac{1}{4}\sum_{ij} A_{ij} - \frac{1}{4}\sum_{ij} A_{ij} z_j  - \frac{1}{4}\sum_{ij} A_{ij} z_i + \frac{1}{4}\sum_{ij} A_{ij} z_i z_j \\
&= \frac{1}{4} \sum_i (2b_i+A_{ii}) + \frac{1}{4}\sum_{ij} A_{ij} + \frac{1}{4} \sum_i \left(-2 b_i  - \sum_{j} A_{ij} - \sum_{j} A_{ji} \right) z_i + \frac{1}{4}\sum_{i \ne j} A_{ij} z_i z_j  \\
&= d + \frac{1}{4}\left(\sum_i  c_i z_i + \sum_{i \ne j} A_{ij} z_i z_j \right) \\
\end{split}
$$

Quantum Hamiltonian:
$$H = \sum_i  c_i Z_i + \sum_{i \ne j} A_{ij} Z_i Z_j = \sum_i  c_i Z_i + \sum_{i < j} (A_{ij} + A_{ji}) Z_i Z_j $$


In [1]:
import numpy as np
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import Statevector, Pauli, SparsePauliOp
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from scipy.optimize import minimize

## Define Problem

In [2]:
n = 6
b = np.random.uniform(-1, 1, (n))
A = np.random.uniform(-1, 1, (n, n))

## Classial Solution

In [3]:
def cost_func(x):
    return np.sum(x * b) + np.sum(np.outer(x, x) * A)

def bin_string_to_x(bin_string):
    return np.array([int(i) for i in bin_string]) #[:, None]

def int_to_cost(i_sol):
    x = bin_string_to_x(np.binary_repr(i_sol, n))
    return cost_func(x)

In [4]:
costs = [int_to_cost(i) for i in range(2**n)]

for i in range(2**n):
    print(i, costs[i])

best_sol = np.argmin(costs)
print(f"\nBest solution: {best_sol}, ({costs[best_sol]})")

0 0.0
1 -0.0019059091405957407
2 -0.631754678586187
3 -1.2643330022235124
4 0.719515500464907
5 -1.0575559317940533
6 0.35086997921824215
7 -2.0568738675374476
8 0.2512306561634279
9 0.8906148560579801
10 -0.7303493728018138
11 -0.7216375874039911
12 2.2199796025927254
13 1.0841982793689127
14 1.5015087309670052
15 -0.2649450067535364
16 -0.4181444312029017
17 0.2162068154358292
18 0.0025748390026634205
19 0.006253671144664796
20 0.4622476132487212
21 -0.6785666632309124
22 1.1460760407938084
23 -0.6254106501825547
24 -0.6736811362236321
25 0.6019602194502467
26 -0.6027872163971217
27 0.042181724780027574
28 1.455944354192381
29 0.9564201867478948
30 1.7899474313584132
31 0.6597508494171982
32 1.0678829271612205
33 -0.1498671506465994
34 0.06415209798026966
35 -1.7842703943242797
36 1.482257441399893
37 -1.510658159526291
38 0.7416357695584643
39 -2.8819522458644493
40 2.013349080268438
41 1.4368891114957658
42 0.6597929007084322
43 -0.5473394825609692
44 3.6769570404715006
45 1.325331

## Quantum Solution

In [5]:
c = -2*b - A.sum(0) - A.sum(1)
const = 0.5 * b.sum() + 0.25 * A.sum() + 0.25 * A.trace()

In [6]:
# Construct Hamiltonian
operators = []
coefs = []

for i in range(n):
    op = ['I'] * n
    op[i] = 'Z'
    operators.append(Pauli(''.join(op)))
    coefs.append(c[i])
    
for i in range(n-1):
    for j in range(i+1, n):
        op = ['I'] * n
        op[i] = 'Z'
        op[j] = 'Z'
        operators.append(Pauli(''.join(op)))
        coefs.append(A[i, j] + A[j, i])

hamiltonian = SparsePauliOp(operators, coefs)
hamiltonian

SparsePauliOp(['ZIIIII', 'IZIIII', 'IIZIII', 'IIIZII', 'IIIIZI', 'IIIIIZ', 'ZZIIII', 'ZIZIII', 'ZIIZII', 'ZIIIZI', 'ZIIIIZ', 'IZZIII', 'IZIZII', 'IZIIZI', 'IZIIIZ', 'IIZZII', 'IIZIZI', 'IIZIIZ', 'IIIZZI', 'IIIZIZ', 'IIIIZZ'],
              coeffs=[-1.44089074+0.j, -1.01040212+0.j, -2.23062765+0.j, -1.03194364+0.j,
  1.30040017+0.j,  2.34794666+0.j,  0.50385069+0.j,  0.6942355 +0.j,
 -0.30514099+0.j, -0.37197615+0.j, -1.21584417+0.j, -0.50676736+0.j,
  0.16087654+0.j,  1.05247395+0.j,  0.63625716+0.j,  1.24923345+0.j,
 -0.34982535+0.j,  0.64129011+0.j,  0.26310916+0.j, -1.77516552+0.j,
 -0.63067241+0.j])

In [7]:
hq = const + 0.25*np.diag(hamiltonian.to_matrix())
assert np.allclose(hq, costs)

In [8]:
qc = QuantumCircuit(n)

for i in range(n):
    qc.ry(Parameter(f'theta_{i}'), i)

qc.draw()

┌─────────────┐
q_0: ┤ Ry(theta_0) ├
     ├─────────────┤
q_1: ┤ Ry(theta_1) ├
     ├─────────────┤
q_2: ┤ Ry(theta_2) ├
     ├─────────────┤
q_3: ┤ Ry(theta_3) ├
     ├─────────────┤
q_4: ┤ Ry(theta_4) ├
     ├─────────────┤
q_5: ┤ Ry(theta_5) ├
     └─────────────┘

In [9]:
def cost_func(params):
    """Return estimate of energy from estimator"""
    pub = (qc, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history.append(energy)
    print(f"Current cost: {energy}")

    return energy

In [10]:
estimator = StatevectorEstimator()

cost_history = []
x0 = 2 * np.pi * np.random.rand(n)
res = minimize(cost_func, x0, method="cobyla")

Current cost: -3.9598724104882552
Current cost: -4.048902100831817
Current cost: -5.28101881427792
Current cost: -6.066765527689072
Current cost: -6.604069775345615
Current cost: -8.192947383254563
Current cost: -8.506616555787907
Current cost: -8.552523989023635
Current cost: -7.593820206335344
Current cost: -8.038631878923534
Current cost: -7.551084675884049
Current cost: -7.43157004939017
Current cost: -9.825324435110836
Current cost: -8.497806335232124
Current cost: -10.169452153041446
Current cost: -9.919797764970138
Current cost: -9.41344188821153
Current cost: -10.362419928385439
Current cost: -9.295226662361411
Current cost: -10.196310886480932
Current cost: -11.02992483443623
Current cost: -10.313609679299063
Current cost: -11.347445185092097
Current cost: -11.670822509552528
Current cost: -12.279405625882134
Current cost: -12.299950052538867
Current cost: -11.765827230351979
Current cost: -12.669771758042767
Current cost: -12.930624431726967
Current cost: -12.980326529451727


In [11]:
res

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -13.547391696603702
       x: [ 3.142e+00  3.142e+00  3.142e+00  6.283e+00  6.283e+00
            9.425e+00]
    nfev: 123
   maxcv: 0.0

In [12]:
sv = Statevector(qc.assign_parameters(res.x))
sv.probabilities()

array([6.55162004e-42, 2.30359919e-29, 1.33911605e-32, 4.70843337e-20,
       3.64394005e-33, 1.28123690e-20, 7.44801831e-24, 2.61877960e-11,
       1.57815761e-53, 5.54892157e-41, 3.22566963e-44, 1.13416985e-31,
       8.77754154e-45, 3.08625003e-32, 1.79408248e-35, 6.30812978e-23,
       5.85889693e-52, 2.06003250e-39, 1.19752716e-42, 4.21059610e-30,
       3.25865497e-43, 1.14576775e-30, 6.66051623e-34, 2.34188791e-21,
       1.41129411e-63, 4.96221689e-51, 2.88460957e-54, 1.01425056e-41,
       7.84946485e-55, 2.75993124e-42, 1.60438858e-45, 5.64115165e-33,
       2.50178366e-31, 8.79646069e-19, 5.11351182e-22, 1.79794945e-09,
       1.39146496e-22, 4.89249609e-10, 2.84407985e-13, 9.99999998e-01,
       6.02630936e-43, 2.11889598e-30, 1.23174536e-33, 4.33090990e-21,
       3.35176794e-34, 1.17850697e-21, 6.85083417e-25, 2.40880514e-12,
       2.23726232e-41, 7.86638362e-29, 4.57284436e-32, 1.60784668e-19,
       1.24434105e-32, 4.37519730e-20, 2.54336646e-23, 8.94266896e-11,
      

In [13]:
np.argmax(sv.probabilities())

39